In [1]:
import xarray
import fsspec
import numpy as np
import s3fs
import proplot
import os

from matplotlib import pyplot as plt
import pandas

/tube1/cha674/Anaconda_Install/miniconda/envs/py3/lib/python3.7/_collections_abc.py:841: MatplotlibDeprecationWarning: 
The examples.directory rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2. In the future, examples will be found relative to the 'datapath' directory.
  self[key] = other[key]


In [2]:
import sys
sys.path.append('/tube1/cha674/CARS_2022/cars-v2/wodpy/wodpy')
import wodnc

In [6]:
START_YEAR = 2010
END_YEAR   = 2011

In [ ]:
obs_type = ['ctd','xbt','pfl'] #,'mrb','gld'] #,'drb'] #,'gld']
#obs_type = ['ctd']

base_s3_url = 's3://noaa-wod-pds'

data_container = {}

output_directory = '/tube1/cha674/CARS_2022_ancillary_data/WOD_2018'

for i_year in range(START_YEAR,END_YEAR+1):

    data_container_for_year = {}
    for i_obs_type in obs_type:
        url = base_s3_url + '/' + str(i_year) + '/wod_' + i_obs_type + '_' + str(i_year) + '.nc' #_pfl_2018.nc'
        print(url)
        fs = s3fs.S3FileSystem(anon=True)
        file_obj = fs.open(url)
        ds = xarray.open_dataset(file_obj,engine='h5netcdf')
        data_container_for_year[i_obs_type] = ds
        
        #Write data out
        if not os.path.isdir(os.path.join(output_directory,str(i_year))):
            os.mkdir(os.path.join(output_directory,str(i_year)))
        print(os.path.join(output_directory,str(i_year),'wod_' + i_obs_type + '_' + str(i_year) + '.nc'))
        ds.to_netcdf(os.path.join(output_directory,str(i_year),'wod_' + i_obs_type + '_' + str(i_year) + '.nc'))
        
    data_container[i_year] = data_container_for_year

In [37]:
WOD_directory = '//oa-decadal-climate/work/observations/CARSv2_ancillary/WOD2018'
platforms_to_get = ['ctd']
WOD_file_name_stem = 'wod_'

profile_vars_to_get = ['Oxygen','Temperature','Salinity','z']
point_vars_to_get   = ['lat','lon'] #,'time','wod_unique_cast']
ancillary_vars_to_get = ['origflagset','country','dataset','Access_no','Recorder',
                         'dbase_orig','Platform','Project','WOD_cruise_identifier',
                         'Institute','needs_z_fix','Ocean_Vehicle','Temperature_Instrument']


for i_year in range(START_YEAR,END_YEAR+1):
    print(i_year)
    for i_platform in platforms_to_get:
        
        file_name  = WOD_file_name_stem + i_platform + '_' + str(i_year) + '.nc'
        WOD_ragged_object = wodnc.Ragged(os.path.join(WOD_directory,str(i_year),file_name))
        n_profiles = WOD_ragged_object.ncasts()
        
        max_depth_levels = 0
        
        for i_profile in range(0,n_profiles):
            
            WOD_profile_object = wodnc.ncProfile(WOD_ragged_object,i_profile)
            n_depth_levels  = WOD_profile_object.n_levels()
            
            if n_depth_levels>max_depth_levels:
                max_depth_levels = n_depth_levels
        
         
        #END for i_profile
        vars_in_dataset = list(WOD_ragged_object.variables().keys())
        
        
        
        profile_data_container   = {}
        ancillary_data_container = {}
        attributes_data_container = {}
        
        for i_var in profile_vars_to_get:
            if i_var in vars_in_dataset:

                profile_data_container[i_var] = np.nan*np.zeros([n_profiles,max_depth_levels],dtype='float32')
                profile_data_container[i_var + '_WODflag'] = np.nan*np.zeros([n_profiles,max_depth_levels],dtype='int8')
                profile_data_container[i_var + '_origflag'] = np.nan*np.zeros([n_profiles,max_depth_levels],dtype='float32')
                
                attributes_data_container[i_var]            = {}
                attributes_data_container[i_var + '_WODflag']  = {}
                attributes_data_container[i_var + '_origflag'] = {}
        #END for i_var

        
        
        point_data_container = {}
        for i_var in point_vars_to_get:
            point_data_container[i_var] = np.nan*np.zeros([n_profiles],dtype='float32')
            
            attributes_data_container[i_var] = {}

        point_data_container['time']            = np.zeros([n_profiles],dtype='datetime64[ns]')
        point_data_container['wod_unique_cast'] = np.zeros([n_profiles],dtype='int32')
        
        for i_ancillary_var in ancillary_vars_to_get:
            if i_ancillary_var in vars_in_dataset:
                
                ancillary_data_container[i_ancillary_var]  = []
                attributes_data_container[i_ancillary_var] = {}

        
        
        #Begin the loop through the profiles
        
        
        for i_profile in range(0,n_profiles):
            
            if (i_profile % 1000) ==0:
                print("Profile: ", i_profile)
            
            WOD_profile_object   = wodnc.ncProfile(WOD_ragged_object,i_profile)                
            
            for i_var in profile_data_container:
                current_variable     = WOD_profile_object.level_unpack(i_var)
                 
                if current_variable.size != 0:
                    #Test for missing data on profile
                    profile_data_container[i_var][i_profile,0:current_variable.size] = current_variable
                    
                    
                #END if current_variable.size != 0:
            if i_profile ==0:
                for i_var in profile_data_container:
                    variable_attributes = WOD_profile_object.show_variable_attr(i_var)
                    for i_attribute in variable_attributes:
                        attributes_data_container[i_var][i_attribute] = WOD_profile_object.get_variable_attr(i_var,i_attribute)
                    #END for i_attribute
                #for i_var 
            #if i_profile ==0 
            

            
            #END for i_var
            
            point_data_container['wod_unique_cast'][i_profile] = WOD_profile_object.metadata('wod_unique_cast')
            point_data_container['time'][i_profile] = np.datetime64(WOD_profile_object.datetime())
            for i_var in point_vars_to_get:
                if i_var in vars_in_dataset:
                    point_data_container[i_var][i_profile] = WOD_profile_object.metadata(i_var)
                    
                    if i_profile==0:
                        variable_attributes = WOD_profile_object.show_variable_attr(i_var)
                        for i_attribute in variable_attributes:
                            
                            attributes_data_container[i_var][i_attribute] = WOD_profile_object.get_variable_attr(i_var,i_attribute)
                #END if i_var
            #END for i_var
            
            for i_ancillary_var in ancillary_data_container.keys():
                ancillary_data_container[i_ancillary_var].append(WOD_profile_object.metadata(i_ancillary_var))
                if i_profile==0:
                    variable_attributes = WOD_profile_object.show_variable_attr(i_ancillary_var)
                    for i_attribute in variable_attributes:

                        attributes_data_container[i_ancillary_var][i_attribute] = WOD_profile_object.get_variable_attr(i_ancillary_var,i_attribute)
                    #END for i_attribute
                #END if i_profile
            #END for i_ancillary
        #END for i_profile
        
        
       
    
        
        dsad
        
            

        
        cast_index = np.arange(1,n_profiles+1)
        
        
        
        CODA_output_dataset = xarray.DataArray(point_data_container['wod_unique_cast'],dims=['cast'],coords={'cast':cast_index}).to_dataset(name='WOD_id') #attrs = attribute_container_for_year['lat']
        #                                         ).to_dataset(name='latitude')
        
        CODA_output_dataset['time'] = xarray.DataArray(point_data_container['time'],dims=['cast'],coords={'cast':cast_index})
        CODA_output_dataset['time'].encoding['units'] = 'seconds since 1980-01-01'
        CODA_output_dataset['lat'] = xarray.DataArray(point_data_container['lat'],dims=['cast'],coords={'cast':cast_index})
        CODA_output_dataset['lon'] = xarray.DataArray(point_data_container['lon'],dims=['cast'],coords={'cast':cast_index})

        

        for i_var in profile_data_container.keys():
            CODA_output_dataset[i_var]  = xarray.DataArray(profile_data_container[i_var],dims=['cast','z_index'],
                                                            coords= {'cast':cast_index,'z_index':np.arange(0,max_depth_levels,1)},
                                                            )
        dsada

2010
Profile:  0
Profile:  1000
Profile:  2000
Profile:  3000
Profile:  4000
Profile:  5000
Profile:  6000
Profile:  7000
Profile:  8000
Profile:  9000
Profile:  10000
Profile:  11000
Profile:  12000
Profile:  13000
Profile:  14000
Profile:  15000
Profile:  16000
Profile:  17000
Profile:  18000
Profile:  19000
Profile:  20000
Profile:  21000
Profile:  22000
Profile:  23000
Profile:  24000
Profile:  25000
Profile:  26000
Profile:  27000
Profile:  28000


NameError: name 'dsad' is not defined

In [40]:
attributes_data_container['Temperature'].keys()


dict_keys(['_FillValue', 'long_name', 'standard_name', 'units', 'grid_mapping', 'ancillary_variables', 'coordinates'])

In [32]:
ancillary_data_container['origflagset']

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'WOCE',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'WOCE',
 '',
 '',
 'WOCE',
 'WOCE',
 'WOCE',
 'WOCE',
 'WOCE',
 '',
 '',
 'WOCE',
 '',
 '',
 'WOCE',
 '',
 '',
 '',
 '',
 'WOCE',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'WOCE',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'WOCE',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'WOCE',
 ''